# BPS Bus Route Optimization

### Problem Statement
The Boston Public Schools has released an RFP seeking to find a solution to optimize their strategy for student transportation to and from school. 

From BPS: "In [2016], transportation costs accounted for \$110 million or 11% of the district’s budget. On a per pupil basis, BPS’s transportation cost is the second highest and more than five times the average of the largest 200 public school districts."

In order to make their transporation network more efficient and potentially less costly, BPS has posed the following two challenges.

#### Step 1: Optimize Bus Stop Locations 

We are working to consolidate bus stops to increase efficiency and decrease student ride time, while also ensuring that students are able to walk safely and quickly to a nearby bus stop. Based on where students live, their destination schools, and the type of service required (i.e. Corner Stop, Door to Door, Wheelchair) how will you assign students to a stop?

#### Step 2: Improve Routing Efficiencies

We are working to get all students to school as quickly as possible in time for the start of the day. Given the location of bus stops, how would you connect these stops to schools over the course of a day creating a full route for a bus that starts and ends at a given bus yard? How will you account for different types of services required (Corner Stop, Door to Door, Wheelchair)?"



### Outcome
The expected outcome of this project is a modeled system of a minimized number of bus stops and routes that provide efficient transportation for every student to and from school. The system will be measured by comparison to the baseline of the existing BPS transportation system as described below:

Scenario | # of required busses |Service hours | Service miles
---| ---| ---
current BPS system | 630 | 3,600 | 44,500

### Approach

1 - Bus Stop Optimization: I hypothesize that by reducing the number of bus stops, the number of service hours will reduce, creating a more efficient system. In order to model this, I will look to highly dense areas and students' maximum walking distance to see if any walking radii overlap. If a student's max walking distance overlaps with that of another student, there is a potential for stops to be combined.In order to measure this, I will need to compare my model's stop count to that of the current BPS system.


2 - Once I have a list of optimized bus stops for each student, I will need to model the bus routes taken in order to serve each student. To model this, I will look to various algorithms to solve what is commonly known as the 'Traveling Salesman Problem"

    * Random routing algorithm
    * Greedy Algorithm
    * 2-Opt Swap on the Greedy Algorithm Route
    
For the purposes of this project, I will only explore the routing for 1 bus serving 1 school. As a continuation of this project, I will look to extend these techniques to the larger system of the Boston School District

### The Data\*

BPS has made available a dataset capturing information for 90 schools and nearly 5000 students. The feautres included in the dataset are as follows:

Variable | Description | Type of Variable
---| ---| ---
Street Number | House number of student | continuous integer
Street Name |  Name of the street on which the student lives | categorical string
Zip Code | Zip Code in which the student lives | categorical integer 
Latitude | Geodetic coordinate system in decimal degrees | continuous float
Longitude | Geodetic coordinate system in decimal degrees | continuous float
Pickup Type | Corner or Door-to-Door | categorical string
Grade | Grade of the student | categorical integer
Geocode | BPS tracks various neighborhoods by census ‘geocodes’ which we use to analyze student neighborhoods | continuous integer
Neighborhood Safety Score | Based on amount of reported violent crime in neighborhood -- 7 = high level and 1 represents a low level | categorical integer
MaxWalk | Maximum distance student should walk to a stop. Actual walking distance along a common path. Each student will have specific max walking distance based on their grade and rates of reported crime within neighborhood | continuous float
Assigned School | School student is required to be transported to and from each day | categorical string
Current School Start | Arrive at school between 10-15 minutes before the start of school. While the bus needs to arrive 10-15 minutes before the start of school, the bus should plan to leave at the start of school. This allows time for buses to unload and builds in “recovery time” to account for unexpected delays on the route | time
Current School End | Time between school start time and school end time is the length of the school day for that school. Assume all students go home at the end of the day (i.e., they attend no after school activities). As detailed in Appendix 1: Rules of the Road buses should arrive by the school’s end time and leave no later than 15 minutes afterward | time
School Address | Full address of school | categorical string
School Lat | Geodetic coordinate system in decimal degrees | continuous float
School Long | Geodetic coordinate system in decimal degrees | continuous float


\* The data provided by BPS is not acutal student information. In order to protect students' safety and privay the data provided by the BPS is only representative of student information.

### Initial Setup and Package Import

In [22]:
from __future__ import print_function

import matplotlib.pyplot as plt
import matplotlib.cm
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import statsmodels.api as sm
from sklearn import metrics
import sklearn as sk

import warnings
warnings.filterwarnings("ignore")

from geopy.distance import vincenty

%matplotlib inline

### Load Data

Student Information

In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/tylershannon/BPS-BusRouteOptimization/master/Data/StopDataModified.csv')

Bus Information

In [24]:
df_busyards = pd.read_csv('https://raw.githubusercontent.com/tylershannon/BPS-BusRouteOptimization/master/Data/BPS-BusYards.csv')

# Student Bus Assignments

In [25]:
df.head()

,Unnamed: 0,StreetNum,StreetName,ZipCode,Lat,Long,PickupType,Grade,Geocode,NeighborhoodSafetyScore,MaxWalk,AssignedSchool,CurrentSchoolStart,CurrentSchoolEnd,SchoolAddress,SchoolLat,SchoolLong,StopFlex,NewLat,NewLong
0,0,1,CLAREMONT PK,2118,42.342049,-71.080765,Corner,3,25025070800,3,0.5,Johnny Pesky School,8:30:00 AM,4:00:00 PM,"236 Beech St, 02131",42.281233,-71.141075,0.5,42.33743,-71.07521
1,1,1,DUNFORD ST,2119,42.317420,-71.094550,Corner,1,25025081500,5,0.4,Tris Speaker School,7:30:00 AM,2:10:00 PM,"40 Pershing Rd, 02130",42.318999,-71.114084,0.4,42.31448,-71.09417
2,2,1,ORGAN PARK ST,2131,42.288240,-71.113130,D2D,6,25025110301,4,0.5,John Gorman School,8:30:00 AM,4:00:00 PM,"110 Beachview Rd, 02128",42.391629,-71.005053,0.0,42.28824,-71.11313
3,3,2295,CENTRE ST,2132,42.271750,-71.161770,Corner,1,25025130402,2,0.4,Roger Clemens School,7:30:00 AM,3:00:00 PM,"200 Heath St, 02130",42.326009,-71.106552,0.4,42.26925,-71.16656
4,4,1,WESTINGHOUSE PZ,2136,42.359400,-71.058700,D2D,4,25025030300,7,0.3,Joe Dobson School,8:30:00 AM,4:00:00 PM,"61 School St, 02119",42.313946,-71.097867,0.0,42.35940,-71.05870


In [26]:
df_busyards.head()

,Bus Number,Bus Yard,Bus Type,Bus Yard Address,Bus Yard Latitude,Bus Yard Longitude
0,B286,Frpt,Full,"2000 Washington Street, Boston, MA, 02119",42.333189,-71.080309
1,B287,Frpt,Full,"2000 Washington Street, Boston, MA, 02119",42.333189,-71.080309
2,B288,Frpt,Full,"2000 Washington Street, Boston, MA, 02119",42.333189,-71.080309
3,B289,Frpt,Full,"2000 Washington Street, Boston, MA, 02119",42.333189,-71.080309
4,B290,Frpt,Full,"2000 Washington Street, Boston, MA, 02119",42.333189,-71.080309


In [27]:
bus_yard_long = df_busyards['Bus Yard Longitude'].unique()
bus_yard_lat = df_busyards['Bus Yard Latitude'].unique()
SchoolToShow = 'Reggie Smith School'

In [28]:
df_bus = df_busyards.copy(deep=True)
df_bus.drop_duplicates(subset = 'Bus Yard Address', inplace = True)
df_bus.drop(df_bus.columns[0:4], axis = 1, inplace = True)
df_bus.reset_index(inplace = True)
df_bus.drop(df_bus.columns[0], axis = 1, inplace = True)
df_bus

,Bus Yard Latitude,Bus Yard Longitude
0,42.333189,-71.080309
1,42.307069,-71.057577
2,42.237390,-71.139795
3,42.382205,-71.075801


# Traveling Salesman Problem

### Setup variables and node lists

### TSP - Simulated Annealing

<img src="SA-Pseudocode.png",width=800,height=800>

In [264]:
from random import random
from random import randint
import math
#from bigfloat import *

def round_figures(x, n):
    """Returns x rounded to n significant figures."""
    return round(x, int(n - math.ceil(math.log10(abs(x)))))

#define function for cost. In this case, cost is the distance traveled
def cost(nodes, debug = False):
    cost = 0
    for a in range(len(nodes)-1):
        current_node = nodes[a]
        next_node = nodes[a+1]
        base_coord = current_node[1], current_node[2]
        local_test = next_node[1], next_node[2]
        dist_test = vincenty(base_coord, local_test).miles
        cost = cost + dist_test
    if debug == True:
        print(current_node)
    return cost 
    
#define function for generating a random neighboring solution
def neighbor(nodes):
    test_list = nodes
    index_a = randint(1,len(test_list)-2)
    index_b = index_a +1 #randint(1,len(test_list)-2)
    test_list[index_a], test_list[index_b] = test_list[index_b], test_list[index_a]
    return test_list
    
#define function for acceptance probability
def acceptance_probability(old_cost, new_cost, temp):
    e = 2.71828
    exponent = round_figures((old_cost - new_cost)/temp, 3)
    a = e**(exponent)
    return a
        
#define function for simulated annealing
def anneal(solution):
    old_cost = cost(solution)
    T = 1.0 #temperature
    T_min = 0.00001
    alpha = 0.9 #temperature reduction coefficient
    while T > T_min:
        i = 1
        while i <= 100:
            new_solution = neighbor(solution)
            new_cost = cost(new_solution)
            ap = acceptance_probability(old_cost, new_cost, T)
            if ap > random():
                solution = new_solution
                old_cost = new_cost
            i += 1
        T = T*alpha
        #print(old_cost, T)
    return solution, old_cost

In [265]:
#-----------------------setup---------------------------
unchecked_nodes_lat = df.loc[df['AssignedSchool'] == SchoolToShow].NewLat.astype(float)
unchecked_nodes_long = df.loc[df['AssignedSchool'] == SchoolToShow].NewLong.astype(float)
index = unchecked_nodes_lat.index

#node list: 0 = index and 1 = latitude and 2 = longitude
unchecked_nodes = map(lambda i, a, b: [i, a, b], index, unchecked_nodes_lat, unchecked_nodes_long)
node_start = [0, df_bus['Bus Yard Latitude'][0], df_bus['Bus Yard Longitude'][0]]
node_end = [0, df.loc[df['AssignedSchool'] == SchoolToShow].SchoolLat[index[0]], df.loc[df['AssignedSchool'] == SchoolToShow].SchoolLong[index[0]]]
current_node = node_start
dist = 0.0
checked_nodes = []
debug = False
#-----------------------test---------------------------

In [263]:
#unchecked_nodes, cost = anneal(unchecked_nodes)